In [1]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import IPython as ip
import  os as os

In [2]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [3]:
#for when we save the files
file = input('Selet a folder to save output') ##enter your file path here - the file is in the repo "summary_stats.csv".

*******************************************************************
*******************************************************************
LET'S LOOK AT SILO FIRST

In [4]:
#SILO subgraph: https://gateway.thegraph.com/api/[api-key]/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB
silo = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/6rcT2GzBXGoi3ZMus92LHajM2b9qWGj7Z3dDhPxoUogB')

##this captures the list of people who borrowed XAI against gOHM
borrows = silo.Query.borrows(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

borrowsDb = sg.query_df([
    borrows.id
    ,borrows.hash
    ,borrows.logIndex
    ,borrows.blockNumber
    ,borrows.timestamp
    ,borrows.account
    ,borrows.market
    ,borrows.market.rates
    ,borrows.asset
    ,borrows.amount
    ,borrows.amountUSD 
])
borrowsDb.head(100)

#this, in turn, pulls repayment events
repays = silo.Query.repays(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #skip = skipValue,
    where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' }

)

repaysDb = sg.query_df([
    repays.id
    ,repays.hash
    ,repays.logIndex
    ,repays.blockNumber
    ,repays.timestamp
    ,repays.account
    ,repays.market
    ,repays.market.rates
    ,repays.asset
    ,repays.amount
    ,repays.amountUSD
    ,repays.position
])
repaysDb.head(100)

In [5]:
##this captures the ENTIRE list of people who borrowed XAI against gOHM
borrowTicker = 0
borrowslist = pd.DataFrame()
borrowsDb = pd.DataFrame()
borrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (borrowTicker)*(1000)
    borrows = silo.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    borrowslist = sg.query_df([
        borrows.id
        ,borrows.hash
        ,borrows.logIndex
        ,borrows.blockNumber
        ,borrows.timestamp
        ,borrows.account
        ,borrows.market
        ,borrows.market.rates
        ,borrows.asset
        ,borrows.amount
        ,borrows.amountUSD
        ,borrows.position
    ])

    #borrowslist.sort_values(by=['borrows_timestamp'])

    borrowsDb=pd.concat([borrowsDb, borrowslist])
    recordTimestamp1 = borrowsDb.iat[borrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    borrowsDbLength = len(borrowsDb)
    borrowListLength = len(borrowslist)
    recordID = borrowsDb.iat[borrowTicker,0]
    borrow = borrowsDb.iat[borrowTicker,1]
    print("iterations: ", borrowTicker, "Lines skipped: ",skipValue, "records collected: ", borrowListLength, " - latest recordID: ",borrow, " ", recordID , " - borrows DB length: ", borrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", borrowTicker, "records collected: ", borrowListLength, " - userdb length: ", borrowsDbLength)
    borrowTicker = borrowTicker+1

    if daysAgo>daysLimit: exit =True
    if borrowListLength<1000: exit =True

borrowsDb.columns = borrowsDb.columns.str.replace("borrows", "transaction")
borrowsDb['event']='borrow'
print("Done. Total Records collected: ", borrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  96  - latest recordID:  0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e0445d6b6e8045f8daf92657   0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e0445d6b6e8045f8daf92657-105  - borrows DB length:  96  - latest record from:  2023-02-09 01:23:59 3 days ago
Done. Total Records collected:  96


In [6]:
#I swear I will figure out synthetic fields, but until then...
borrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From borrowsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()
borrowsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,BORROWER,True,1.860000e+23,0,0,1,0,0,borrow,2023-02-09 06:23:59
1,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,BORROWER,True,1.860000e+23,0,0,1,0,0,borrow,2023-02-09 06:23:59
2,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,BORROWER,True,1.860000e+23,0,0,1,0,0,borrow,2023-02-09 06:23:59
3,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,BORROWER,True,1.860000e+23,0,0,1,0,0,borrow,2023-02-09 06:23:59
4,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,0x16f2ff52f7a9e9057848569ea37c3e59eb558af0e044...,105,16589389,1675923839,0xb0399dac8f4d4100b49b2a7b3873481114229d18,2,2,0,2,...,BORROWER,True,1.860000e+23,0,0,1,0,0,borrow,2023-02-09 06:23:59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,117,16034628,1669229867,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0.000000e+00,0,0,1,1,0,borrow,2022-11-23 18:57:47
92,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,117,16034628,1669229867,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0.000000e+00,0,0,1,1,0,borrow,2022-11-23 18:57:47
93,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,117,16034628,1669229867,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0.000000e+00,0,0,1,1,0,borrow,2022-11-23 18:57:47
94,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,0x83addaa30259315b10869fa8653a468eb1cc82a43f57...,117,16034628,1669229867,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,BORROWER,True,0.000000e+00,0,0,1,1,0,borrow,2022-11-23 18:57:47


In [7]:
path =file+'/siloFinance_borrows_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(len(borrowsDbTimed))+'_records.csv'
borrowsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_borrows_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(len(borrowsDbTimed))+'_records_piped.csv'
borrowsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [8]:
##this tracks the ENTIRE list of events for repaying the XAI repayed against gOHM
repayTicker = 0
repayslist = pd.DataFrame()
repaysDb = pd.DataFrame()
repayListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (repayTicker)*(1000)
    repays = silo.Query.repays(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    repayslist = sg.query_df([
        repays.id
        ,repays.hash
        ,repays.logIndex
        ,repays.blockNumber
        ,repays.timestamp
        ,repays.account
        ,repays.market
        ,repays.market.rates
        ,repays.asset
        ,repays.amount
        ,repays.amountUSD
        ,repays.position
    ])

    #repayslist.sort_values(by=['repays_timestamp'])

    repaysDb=pd.concat([repaysDb, repayslist])
    recordTimestamp1 = repaysDb.iat[repayTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    repaysDbLength = len(repaysDb)
    repayListLength = len(repayslist)
    recordID = repaysDb.iat[repayTicker,0]
    repay = repaysDb.iat[repayTicker,1]
    print("iterations: ", repayTicker, "Lines skipped: ",skipValue, "records collected: ", repayListLength, " - latest recordID: ",repay, " ", recordID , " - repays DB length: ", repaysDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", repayTicker, "records collected: ", repayListLength, " - userdb length: ", repaysDbLength)
    repayTicker = repayTicker+1

    if daysAgo>daysLimit: exit =True
    if repayListLength<1000: exit =True

repaysDb.columns = repaysDb.columns.str.replace("repays", "transaction")
repaysDb['event']='repay'
print("Done. Total Records collected: ", repaysDbLength)

iterations:  0 Lines skipped:  0 records collected:  12  - latest recordID:  0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad01a68715d0385e8b26cd   0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad01a68715d0385e8b26cd-242  - repays DB length:  12  - latest record from:  2022-11-25 16:03:47 79 days ago
Done. Total Records collected:  12


In [9]:
repaysDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
1,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
2,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
3,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
4,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
5,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,0x0c0283dc0e75f17b5e9eb79b05bea3cae12d32204aad...,242,16049560,1669410227,0x7776421dd146dfd6ea947cbf29b69c519807d259,26,2,24,18,...,1669410227,BORROWER,True,0,0,0,1,1,0,repay
6,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay
7,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay
8,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay
9,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,0x748852c58dd4b57d3f6ed80bc81f74b439efcb94968b...,200,16036129,1669248167,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,2,0,2,2,...,1669248167,BORROWER,True,0,0,0,1,1,0,repay


In [10]:
#this tracks the ENTIRE list of events for depositing gOHM for lending
depositTicker = 0
depositslist = pd.DataFrame()
depositsDb = pd.DataFrame()
depositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (depositTicker)*(1000)
    deposits = silo.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    depositslist = sg.query_df([
        deposits.id
        ,deposits.hash
        ,deposits.logIndex
        ,deposits.blockNumber
        ,deposits.timestamp
        ,deposits.account
        ,deposits.market
        ,deposits.market.rates
        ,deposits.asset
        ,deposits.amount
        ,deposits.amountUSD
        ,deposits.position
    ])

    #depositslist.sort_values(by=['deposits_timestamp'])

    depositsDb=pd.concat([depositsDb, depositslist])
    recordTimestamp1 = depositsDb.iat[depositTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    depositsDbLength = len(depositsDb)
    depositListLength = len(depositslist)
    recordID = depositsDb.iat[depositTicker,0]
    deposit = depositsDb.iat[depositTicker,1]
    print("iterations: ", depositTicker, "Lines skipped: ",skipValue, "records collected: ", depositListLength, " - latest recordID: ",deposit, " ", recordID , " - deposits DB length: ", depositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", depositTicker, "records collected: ", depositListLength, " - userdb length: ", depositsDbLength)
    depositTicker = depositTicker+1
    if daysAgo>daysLimit: exit =True
    if depositListLength<1000: exit =True

#depositsDb.sort_values(by=['deposits_timestamp'])

depositsDb.columns = depositsDb.columns.str.replace("deposits", "transaction")
depositsDb['event']='deposit'
print("Done. Total Records collected: ", depositsDbLength)




iterations:  0 Lines skipped:  0 records collected:  228  - latest recordID:  0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706424187f40001539f1395   0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706424187f40001539f1395-262  - deposits DB length:  228  - latest record from:  2023-02-10 17:58:11 2 days ago
Done. Total Records collected:  228


In [11]:
#I swear I will figure out synthetic fields, but until then...
depositsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From depositsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()
depositsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
1,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
2,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
3,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
4,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x976fb2b1e49590c04707140f0a1bce4465fda42dfe9c...,0x976fb2b1e49590c04707140f0a1bce4465fda42dfe9c...,283,16505276,1674908423,0x42daeb9ca091e3933c8ee93c029701b926b1f336,2,2,0,1,...,LENDER,True,5.886017e+18,1,0,0,0,0,deposit,2023-01-28 12:20:23
96,0x7811e7757339551966af37751af4b56c16a3e141d682...,0x7811e7757339551966af37751af4b56c16a3e141d682...,149,16366137,1673229911,0x25fdda95197f804316f5ba8dbb66ac173ee19981,1,0,1,1,...,LENDER,True,0.000000e+00,1,1,0,0,0,deposit,2023-01-09 02:05:11
97,0x7811e7757339551966af37751af4b56c16a3e141d682...,0x7811e7757339551966af37751af4b56c16a3e141d682...,149,16366137,1673229911,0x25fdda95197f804316f5ba8dbb66ac173ee19981,1,0,1,1,...,LENDER,True,0.000000e+00,1,1,0,0,0,deposit,2023-01-09 02:05:11
98,0x7811e7757339551966af37751af4b56c16a3e141d682...,0x7811e7757339551966af37751af4b56c16a3e141d682...,149,16366137,1673229911,0x25fdda95197f804316f5ba8dbb66ac173ee19981,1,0,1,1,...,LENDER,True,0.000000e+00,1,1,0,0,0,deposit,2023-01-09 02:05:11


In [12]:
path =file+'/siloFinance_deposits_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(len(depositsDbTimed))+'_records.csv'
depositsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_deposits_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(len(depositsDbTimed))+'_records_piped.csv'
depositsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [13]:
print("depoosit rows ",len(depositsDb) ,", borrow rows ", len(borrowsDb),", repay rows ", len(repaysDb), ". Expected rows: ",(len(depositsDb)+len(borrowsDb))+len(repaysDb) )

depoosit rows  228 , borrow rows  96 , repay rows  12 . Expected rows:  336


In [14]:
siloEventsDb = pd.DataFrame()
siloEventsDb=pd.concat([depositsDb, borrowsDb,repaysDb])
siloEventsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,NaN,LENDER,True,1,1,1,0,0,0,deposit
1,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,NaN,LENDER,True,1,1,1,0,0,0,deposit
2,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,NaN,LENDER,True,1,1,1,0,0,0,deposit
3,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,NaN,LENDER,True,1,1,1,0,0,0,deposit
4,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,NaN,LENDER,True,1,1,1,0,0,0,deposit
5,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,NaN,LENDER,True,1,1,1,0,0,0,deposit
6,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
7,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
8,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit
9,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,0x7c0686952bdf22de3c8ce87e732690a4365c23f25939...,204,16592784,1675964891,0xc04f84a02cc65f14f4e8c982a7a467ee88c5311e,27,9,18,35,...,NaN,LENDER,True,80385198564684193341455,1,0,0,0,0,deposit


In [15]:
len(siloEventsDb)

336

In [16]:
#I swear I will figure out synthetic fields, but until then...
siloEventsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From siloEventsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()
siloEventsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
1,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
2,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
3,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
4,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,0xff978d5e58c65cbc9d73fbf58f44166724c2db0f0706...,262,16601490,1676069891,0xa7daeb7fc210f934508c8c7ce833a2cdd6630fb3,1,1,0,1,...,LENDER,True,1.000000e+00,1,1,0,0,0,deposit,2023-02-10 22:58:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x976fb2b1e49590c04707140f0a1bce4465fda42dfe9c...,0x976fb2b1e49590c04707140f0a1bce4465fda42dfe9c...,283,16505276,1674908423,0x42daeb9ca091e3933c8ee93c029701b926b1f336,2,2,0,1,...,LENDER,True,5.886017e+18,1,0,0,0,0,deposit,2023-01-28 12:20:23
96,0x7811e7757339551966af37751af4b56c16a3e141d682...,0x7811e7757339551966af37751af4b56c16a3e141d682...,149,16366137,1673229911,0x25fdda95197f804316f5ba8dbb66ac173ee19981,1,0,1,1,...,LENDER,True,0.000000e+00,1,1,0,0,0,deposit,2023-01-09 02:05:11
97,0x7811e7757339551966af37751af4b56c16a3e141d682...,0x7811e7757339551966af37751af4b56c16a3e141d682...,149,16366137,1673229911,0x25fdda95197f804316f5ba8dbb66ac173ee19981,1,0,1,1,...,LENDER,True,0.000000e+00,1,1,0,0,0,deposit,2023-01-09 02:05:11
98,0x7811e7757339551966af37751af4b56c16a3e141d682...,0x7811e7757339551966af37751af4b56c16a3e141d682...,149,16366137,1673229911,0x25fdda95197f804316f5ba8dbb66ac173ee19981,1,0,1,1,...,LENDER,True,0.000000e+00,1,1,0,0,0,deposit,2023-01-09 02:05:11


In [17]:
siloEventsDblength = len(siloEventsDbTimed)
siloEventsDblength

336

In [18]:
path =file+'/siloFinance_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records.csv'
siloEventsDbTimed.to_csv(path, index = False)

path =file+'/siloFinance_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloEventsDblength)+'_records_piped.csv'
siloEventsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [19]:
#let's pull interest rate data
interestRateTicker = 0
interestRateslist = pd.DataFrame()
interestRatesDb = pd.DataFrame()
interestRateListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (interestRateTicker)*(1000)
    interestRates = silo.Query.interestRates(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    interestRateslist = sg.query_df([
        interestRates.id
        ,interestRates.rate #5.21% should be stored as 5.21
        ,interestRates.duration #in days
        ,interestRates.maturityBlock #Maturity of the loan in block height. Only applies to fixed term lending (e.g. Notional)
        ,interestRates.side #The party the interest is paid to / received from
        ,interestRates.type #The type of interest rate (e.g. stable, fixed, variable, etc)
        ,interestRates.market
        ,interestRates.marketAsset
        ,interestRates.token
    ])

    #interestRateslist.sort_values(by=['interestRates_timestamp'])

    interestRatesDb=pd.concat([interestRateslist, interestRatesDb])
    interestRatesDbLength = len(interestRatesDb)
    interestRateListLength = len(interestRateslist)
    print("iterations: ", interestRateTicker, "Lines skipped: ",skipValue, "records collected: ", interestRateListLength)
    #print("iterations: ", interestRateTicker, "records collected: ", interestRateListLength, " - userdb length: ", interestRatesDbLength)
    interestRateTicker = interestRateTicker+1
    if interestRateListLength<1000: exit =True

print("Done. Total Records collected: ", interestRatesDbLength)




iterations:  0 Lines skipped:  0 records collected:  6
Done. Total Records collected:  6


In [20]:
interestRatesDb.head(10)

,interestRates_id,interestRates_rate,interestRates_duration,interestRates_maturityBlock,interestRates_side,interestRates_type,interestRates_market_id,interestRates_market_protocol_id,interestRates_market_name,interestRates_market_isActive,...,interestRates_token_symbol,interestRates_token_decimals,interestRates_token_type,interestRates_token_lastPriceUSD,interestRates_token_lastPriceBlockNumber,interestRates_token_market_id,interestRates_token_asset_id,interestRates_token_derivativeConversion,interestRates_token_totalSupply,interestRates_token_activeOracle_id
0,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,2.148783,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,XAI,18,Asset,1.002396,16616490,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...
1,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,0.000000,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,WETH,18,Asset,1517.793152,16616490,None,None,1.0,0,None
2,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,0.000000,None,None,LENDER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,gOHM,18,Asset,2648.929943,16616490,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...
3,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,3.277791,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,XAI,18,Asset,1.002396,16616490,None,None,1.0,0,0xe37b8c83138caf12e57632d19c06eb561d47e423-0xd...
4,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,0.000000,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,WETH,18,Asset,1517.793152,16616490,None,None,1.0,0,None
5,BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d...,0.000000,None,None,BORROWER,VARIABLE,0xc413dd03555f3eb29d834b482d386b2999dc2eb0,0xd998c35b7900b344bbbe6555cc11576942cf309d,gOHM,True,...,gOHM,18,Asset,2648.929943,16616490,None,None,1.0,0,0xba53d3a08fb8dd4309881b4e8a3bc386ea0ddb1f-0x0...


In [21]:
interestRatesDbLength = len(interestRatesDb)
path =file+'/siloFinance_interestRates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(interestRatesDbLength)+'_records.csv'
interestRatesDb.to_csv(path, index = False)

path =file+'/siloFinance_interestRates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(interestRatesDbLength)+'_records_piped.csv'
interestRatesDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


********************************************************************
********************************************************************


LET'S LOOK AT VESTA WHILE WE'RE ON A RUN

Annoying: the schema for repay events is completely different from borrow and deposit. Can't union them into a single table without crunching it first. Argh.

Interest Rate data is weird, too. But I may not need it?

In [22]:
#https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance
vesta = sg.load_api('https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance')

In [23]:
##this captures the list of people who vestaBorrowed XAI against gOHM
vestaBorrowTicker = 0
vestaBorrowslist = pd.DataFrame()
vestaBorrowsDb = pd.DataFrame()
vestaBorrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaBorrowTicker)*(1000)
    vestaBorrows = vesta.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaBorrowslist = sg.query_df([
        vestaBorrows.id
        ,vestaBorrows.hash
        ,vestaBorrows.logIndex
        ,vestaBorrows.blockNumber
        ,vestaBorrows.timestamp
        ,vestaBorrows.account
        ,vestaBorrows.market
        ,vestaBorrows.market.rates
        ,vestaBorrows.asset
        ,vestaBorrows.amount
        ,vestaBorrows.amountUSD
        ,vestaBorrows.position
    ])

    #vestaBorrowslist.sort_values(by=['vestaBorrows_timestamp'])

    vestaBorrowsDb=pd.concat([vestaBorrowsDb, vestaBorrowslist])
    recordTimestamp1 = vestaBorrowsDb.iat[vestaBorrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaBorrowsDbLength = len(vestaBorrowsDb)
    vestaBorrowListLength = len(vestaBorrowslist)
    recordID = vestaBorrowsDb.iat[vestaBorrowTicker,0]
    vestaBorrow = vestaBorrowsDb.iat[vestaBorrowTicker,1]
    print("iterations: ", vestaBorrowTicker, "Lines skipped: ",skipValue, "records collected: ", vestaBorrowListLength, " - latest recordID: ",vestaBorrow, " ", recordID , " - vestaBorrows DB length: ", vestaBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaBorrowTicker, "records collected: ", vestaBorrowListLength, " - userdb length: ", vestaBorrowsDbLength)
    vestaBorrowTicker = vestaBorrowTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaBorrowListLength<1000: exit =True

#vestaBorrowsDbTimed.columns = vestaBorrowsDbTimed.columns.str.replace("borrows", "transaction")
vestaBorrowsDb['event']='borrow'
print("Done. Total Records collected: ", vestaBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0x1b54e1c3ef70e868dbce6ee79aded1704c308e4d38f9a62f9802e4d221baf50b   borrow-0x1b54e1c3ef70e868dbce6ee79aded1704c308e4d38f9a62f9802e4d221baf50b-37  - vestaBorrows DB length:  1000  - latest record from:  2023-02-11 20:27:03 1 days ago
iterations:  1 Lines skipped:  1000 records collected:  401  - latest recordID:  0xa8530b80a3aae63661e62d05f9c83afc5e62634b5f4c755d26e7bc39df487a43   borrow-0xa8530b80a3aae63661e62d05f9c83afc5e62634b5f4c755d26e7bc39df487a43-45  - vestaBorrows DB length:  1401  - latest record from:  2023-02-11 14:16:17 1 days ago
Done. Total Records collected:  1401


In [24]:
vestaBorrowsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(borrows_timestamp::int) dateTime "
               "From vestaBorrowsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaBorrowsDbTimed.head(100)

,borrows_id,borrows_hash,borrows_logIndex,borrows_blockNumber,borrows_timestamp,borrows_account_id,borrows_account_positionCount,borrows_account_openPositionCount,borrows_account_closedPositionCount,borrows_account_depositCount,...,borrows_position_side,borrows_position_isCollateral,borrows_position_balance,borrows_position_depositCount,borrows_position_withdrawCount,borrows_position_borrowCount,borrows_position_repayCount,borrows_position_liquidationCount,event,dateTime
0,borrow-0x1b54e1c3ef70e868dbce6ee79aded1704c308...,0x1b54e1c3ef70e868dbce6ee79aded1704c308e4d38f9...,37,60125704,1676165223,0x6ae68eb2fa416fe091803fde014bbf7475468979,6,2,4,4,...,BORROWER,NaN,9.177662e+22,0,0,1,0,0,borrow,2023-02-12 01:27:03
1,borrow-0xa8530b80a3aae63661e62d05f9c83afc5e626...,0xa8530b80a3aae63661e62d05f9c83afc5e62634b5f4c...,45,60064234,1676142977,0xa13f3599ff620500f360dc6185ad30f625ebfc38,2,2,0,1,...,BORROWER,NaN,1.614690e+21,0,0,1,0,0,borrow,2023-02-11 19:16:17
2,borrow-0x0712c2c572f53f272b382289d178c5801205e...,0x0712c2c572f53f272b382289d178c5801205eb0b7c99...,45,59984711,1676119566,0x67fa1561c5f5631de08e38d4484e0df67872f626,2,2,0,1,...,BORROWER,NaN,6.000370e+21,0,0,1,0,0,borrow,2023-02-11 12:46:06
3,borrow-0xfa585079dbbd9dc2a051922131b8d6d5d65bb...,0xfa585079dbbd9dc2a051922131b8d6d5d65bbfa55a6a...,37,59820888,1676068023,0xc04f25ef37e2a960afa400e623c386625c7802e3,2,2,0,1,...,BORROWER,NaN,1.350000e+21,0,0,2,0,0,borrow,2023-02-10 22:27:03
4,borrow-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e...,0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6...,38,59495981,1675974783,0x15bae977db3b62ceb5a67921fae589fd80624e8d,3,2,1,4,...,BORROWER,NaN,5.500004e+23,0,0,4,150,0,borrow,2023-02-09 20:33:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,borrow-0xc502f9e5eb845a34d721e418dd597bb64e15a...,0xc502f9e5eb845a34d721e418dd597bb64e15abace157...,29,48288379,1671802945,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,BORROWER,NaN,0.000000e+00,0,0,1,2,0,borrow,2022-12-23 13:42:25
96,borrow-0xd6835e5cc616cf65b8f46c3fb685c7d436206...,0xd6835e5cc616cf65b8f46c3fb685c7d4362061621bc4...,34,48094577,1671731058,0x804157889ac56a4e50278610c67960c1605a3264,16,2,14,24,...,BORROWER,NaN,1.155859e+22,0,0,11,0,0,borrow,2022-12-22 17:44:18
97,borrow-0xdbceaecbce8edc0d0d07d87f907a46c50c0e3...,0xdbceaecbce8edc0d0d07d87f907a46c50c0e33884ce3...,42,47726191,1671602208,0x73831e7c600f48cebbbff957b466be8e5c087f34,4,3,1,5,...,BORROWER,NaN,6.423430e+21,0,0,1,0,0,borrow,2022-12-21 05:56:48
98,borrow-0x67bfcaa98439f011745fde9e00deb0fb2461b...,0x67bfcaa98439f011745fde9e00deb0fb2461beb1c73a...,34,47689602,1671589080,0xc9e166f2737e004e93800dcef0d608b014df07f6,25,4,21,16,...,BORROWER,NaN,3.515001e+21,0,0,30,21,0,borrow,2022-12-21 02:18:00


In [25]:
vestaBorrowsDbTimedLength = len(vestaBorrowsDbTimed)
path =file+'/vesta_borrow_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaBorrowsDbTimedLength)+'_records.csv'
vestaBorrowsDbTimed.to_csv(path, index = False)

path =file+'/vesta_borrow_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaBorrowsDbTimedLength)+'_records_piped.csv'
vestaBorrowsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [26]:
#this tracks the events for vestaDepositing gOHM for lending 
vestaDepositTicker = 0
vestaDepositslist = pd.DataFrame()
vestaDepositsDb = pd.DataFrame()
vestaDepositListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaDepositTicker)*(1000)
    vestaDeposits = vesta.Query.deposits(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaDepositslist = sg.query_df([
        vestaDeposits.id
        ,vestaDeposits.hash
        ,vestaDeposits.logIndex
        ,vestaDeposits.blockNumber
        ,vestaDeposits.timestamp
        ,vestaDeposits.account
        ,vestaDeposits.market
        ,vestaDeposits.market.rates
        ,vestaDeposits.asset
        ,vestaDeposits.amount
        ,vestaDeposits.amountUSD
        ,vestaDeposits.position
    ])

    #vestaDepositslist.sort_values(by=['vestaDeposits_timestamp'])

    vestaDepositsDb=pd.concat([vestaDepositsDb, vestaDepositslist])
    recordTimestamp1 = vestaDepositsDb.iat[vestaDepositTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaDepositsDbLength = len(vestaDepositsDb)
    vestaDepositListLength = len(vestaDepositslist)
    recordID = vestaDepositsDb.iat[vestaDepositTicker,0]
    vestaDeposit = vestaDepositsDb.iat[vestaDepositTicker,1]
    print("iterations: ", vestaDepositTicker, "Lines skipped: ",skipValue, "records collected: ", vestaDepositListLength, " - latest recordID: ",vestaDeposit, " ", recordID , " - vestaDeposits DB length: ", vestaDepositsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaDepositTicker, "records collected: ", vestaDepositListLength, " - userdb length: ", vestaDepositsDbLength)
    vestaDepositTicker = vestaDepositTicker+1
    if daysAgo>daysLimit: exit =True
    if vestaDepositListLength<1000: exit =True

#vestaDepositsDb.sort_values(by=['vestaDeposits_timestamp'])

vestaDepositsDb.columns = vestaDepositsDb.columns.str.replace("deposits", "transaction")
vestaDepositsDb['event']='deposit'
print("Done. Total Records collected: ", vestaDepositsDbLength)




iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0xa8530b80a3aae63661e62d05f9c83afc5e62634b5f4c755d26e7bc39df487a43   deposit-0xa8530b80a3aae63661e62d05f9c83afc5e62634b5f4c755d26e7bc39df487a43-45  - vestaDeposits DB length:  1000  - latest record from:  2023-02-11 14:16:17 1 days ago
iterations:  1 Lines skipped:  1000 records collected:  335  - latest recordID:  0x0712c2c572f53f272b382289d178c5801205eb0b7c99154dc88b0510191f9c98   deposit-0x0712c2c572f53f272b382289d178c5801205eb0b7c99154dc88b0510191f9c98-45  - vestaDeposits DB length:  1335  - latest record from:  2023-02-11 07:46:06 1 days ago
Done. Total Records collected:  1335


In [27]:
vestaDepositsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,deposit-0xa8530b80a3aae63661e62d05f9c83afc5e62...,0xa8530b80a3aae63661e62d05f9c83afc5e62634b5f4c...,45,60064234,1676142977,0xa13f3599ff620500f360dc6185ad30f625ebfc38,2,2,0,1,...,NaN,LENDER,True,1820578000000000000,1,0,0,0,0,deposit
1,deposit-0x0712c2c572f53f272b382289d178c5801205...,0x0712c2c572f53f272b382289d178c5801205eb0b7c99...,45,59984711,1676119566,0x67fa1561c5f5631de08e38d4484e0df67872f626,2,2,0,1,...,NaN,LENDER,True,5071806000000000000,1,0,0,0,0,deposit
2,deposit-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5...,0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6...,38,59495981,1675974783,0x15bae977db3b62ceb5a67921fae589fd80624e8d,3,2,1,4,...,NaN,LENDER,True,419221347099961371843,1,0,0,0,0,deposit
3,deposit-0x5d3025a16815324cf1afca689903a14d2621...,0x5d3025a16815324cf1afca689903a14d2621c19c9572...,37,59294798,1675915490,0x7723c45fb37092ee65202db7a2fe562c5fe0ab48,9,4,5,5,...,NaN,LENDER,True,395431000000000000,1,0,0,0,0,deposit
4,deposit-0xabfb1cba9108afe5c2dde3f21f5b4de92fae...,0xabfb1cba9108afe5c2dde3f21f5b4de92fae79791af2...,4,58531581,1675694548,0x1d883459bbc71042a9072f106427c4993de44794,2,2,0,2,...,NaN,LENDER,True,18790084000000000515,2,0,0,0,0,deposit
5,deposit-0xdb674a81e3a0fd547a3a2ace9befd38a3b86...,0xdb674a81e3a0fd547a3a2ace9befd38a3b867df74fc0...,30,58285118,1675618839,0x0fd84d7cb911728737556684050782b298f70f0f,7,2,5,5,...,NaN,LENDER,True,139919099999999990809,1,0,0,0,0,deposit
6,deposit-0xe1f1f1cfd372bbe64a06346c1dde1102e155...,0xe1f1f1cfd372bbe64a06346c1dde1102e155be4ec282...,34,58200953,1675595730,0x2682efcc31de7a06ed3bf8b0be156ff88e1663a1,7,2,5,5,...,NaN,LENDER,True,99538985999999994122,2,0,0,0,0,deposit
7,deposit-0x81d0e5f18de8d3995c911b0b022d2b051866...,0x81d0e5f18de8d3995c911b0b022d2b05186668fe6c0b...,23,58147623,1675579717,0x53430bde5e57ed878406a20ca101b0853f173acf,4,2,2,2,...,NaN,LENDER,True,2648200000000000000,1,0,0,0,0,deposit
8,deposit-0x16bfcc7fe7939a6c9935e5062c8067a57acd...,0x16bfcc7fe7939a6c9935e5062c8067a57acd4b32abb1...,42,58118599,1675569579,0xc6f84e7142eeaf930c62584f45ba7106bc3e2d68,2,2,0,1,...,NaN,LENDER,True,314789000000000000,1,0,0,0,0,deposit
9,deposit-0x61a65a717a84989f1315c17975c28846056a...,0x61a65a717a84989f1315c17975c28846056a95bb61c2...,42,58014267,1675533398,0xf5d0318692f822680b733955828a25507f031c95,2,0,2,1,...,1.675979e+09,LENDER,False,0,1,1,0,0,0,deposit


In [28]:
vestaDepositsDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From vestaDepositsDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaDepositsDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,deposit-0xa8530b80a3aae63661e62d05f9c83afc5e62...,0xa8530b80a3aae63661e62d05f9c83afc5e62634b5f4c...,45,60064234,1676142977,0xa13f3599ff620500f360dc6185ad30f625ebfc38,2,2,0,1,...,LENDER,True,1.820578e+18,1,0,0,0,0,deposit,2023-02-11 19:16:17
1,deposit-0x0712c2c572f53f272b382289d178c5801205...,0x0712c2c572f53f272b382289d178c5801205eb0b7c99...,45,59984711,1676119566,0x67fa1561c5f5631de08e38d4484e0df67872f626,2,2,0,1,...,LENDER,True,5.071806e+18,1,0,0,0,0,deposit,2023-02-11 12:46:06
2,deposit-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5...,0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6...,38,59495981,1675974783,0x15bae977db3b62ceb5a67921fae589fd80624e8d,3,2,1,4,...,LENDER,True,4.192213e+20,1,0,0,0,0,deposit,2023-02-09 20:33:03
3,deposit-0x5d3025a16815324cf1afca689903a14d2621...,0x5d3025a16815324cf1afca689903a14d2621c19c9572...,37,59294798,1675915490,0x7723c45fb37092ee65202db7a2fe562c5fe0ab48,9,4,5,5,...,LENDER,True,3.954310e+17,1,0,0,0,0,deposit,2023-02-09 04:04:50
4,deposit-0xabfb1cba9108afe5c2dde3f21f5b4de92fae...,0xabfb1cba9108afe5c2dde3f21f5b4de92fae79791af2...,4,58531581,1675694548,0x1d883459bbc71042a9072f106427c4993de44794,2,2,0,2,...,LENDER,True,1.879008e+19,2,0,0,0,0,deposit,2023-02-06 14:42:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,deposit-0x4de9095f197be41e69b8af94de1c417edb35...,0x4de9095f197be41e69b8af94de1c417edb352c56ee2f...,42,43981545,1670300772,0xc04f25ef37e2a960afa400e623c386625c7802e3,2,2,0,1,...,LENDER,True,1.125000e+18,1,0,0,0,0,deposit,2022-12-06 04:26:12
96,deposit-0xa2658ef3d7462248ddf736dfc67f0546c991...,0xa2658ef3d7462248ddf736dfc67f0546c9913841a632...,4,43909436,1670274054,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,LENDER,False,0.000000e+00,2,2,0,0,0,deposit,2022-12-05 21:00:54
97,deposit-0x92fb45376ce2474a9812168f9be4d401a44c...,0x92fb45376ce2474a9812168f9be4d401a44cb0bd62d9...,23,43909041,1670273939,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,LENDER,False,0.000000e+00,2,2,0,0,0,deposit,2022-12-05 20:58:59
98,deposit-0xf4eb0268999a30bf9a33fc801dfbe2dc65c2...,0xf4eb0268999a30bf9a33fc801dfbe2dc65c2e18ef07f...,4,43845816,1670255076,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,LENDER,False,0.000000e+00,1,1,0,0,0,deposit,2022-12-05 15:44:36


In [29]:
vestaDepositsDbTimedLength = len(vestaDepositsDbTimed)
path =file+'/vesta_deposit_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaDepositsDbTimedLength)+'_records.csv'
vestaDepositsDbTimed.to_csv(path, index = False)

path =file+'/vesta_deposit_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaDepositsDbTimedLength)+'_records_piped.csv'
vestaDepositsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [30]:
##this tracks the ENTIRE list of events for vestaRepaying the VST vestaRepayed against gOHM
vestaRepayTicker = 0
vestaRepayslist = pd.DataFrame()
vestaRepaysDb = pd.DataFrame()
vestaRepayListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaRepayTicker)*(1000)
    vestaRepays = vesta.Query.repays(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaRepayslist = sg.query_df([
        vestaRepays.id
        ,vestaRepays.hash
        ,vestaRepays.logIndex
        ,vestaRepays.blockNumber
        ,vestaRepays.timestamp
        ,vestaRepays.account
        ,vestaRepays.market
        ,vestaRepays.market.rates
        ,vestaRepays.asset
        ,vestaRepays.amount
        ,vestaRepays.amountUSD
        ,vestaRepays.position
    ])

    #vestaRepayslist.sort_values(by=['vestaRepays_timestamp'])

    vestaRepaysDb=pd.concat([vestaRepaysDb, vestaRepayslist])
    recordTimestamp1 = vestaRepaysDb.iat[vestaRepayTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaRepaysDbLength = len(vestaRepaysDb)
    vestaRepayListLength = len(vestaRepayslist)
    recordID = vestaRepaysDb.iat[vestaRepayTicker,0]
    vestaRepay = vestaRepaysDb.iat[vestaRepayTicker,1]
    print("iterations: ", vestaRepayTicker, "Lines skipped: ",skipValue, "records collected: ", vestaRepayListLength, " - latest recordID: ",vestaRepay, " ", recordID , " - vestaRepays DB length: ", vestaRepaysDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaRepayTicker, "records collected: ", vestaRepayListLength, " - userdb length: ", vestaRepaysDbLength)
    vestaRepayTicker = vestaRepayTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaRepayListLength<1000: exit =True

vestaRepaysDb.columns = vestaRepaysDb.columns.str.replace("repays", "transaction")
vestaRepaysDb['event']='Repay'
print("Done. Total Records collected: ", vestaRepaysDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0x851007fa5a8371105a73452ccbd7a0a4eb976e7d2c120726ded1ff1385740cbc   repay-0x851007fa5a8371105a73452ccbd7a0a4eb976e7d2c120726ded1ff1385740cbc-32  - vestaRepays DB length:  1000  - latest record from:  2023-02-10 21:49:48 2 days ago
iterations:  1 Lines skipped:  1000 records collected:  991  - latest recordID:  0xa243d87a35b20328eac5637a6db98058707cd0161df6940c58cca7ea16f24ce7   repay-0xa243d87a35b20328eac5637a6db98058707cd0161df6940c58cca7ea16f24ce7-61  - vestaRepays DB length:  1991  - latest record from:  2023-02-09 16:40:10 3 days ago
Done. Total Records collected:  1991


In [31]:
vestaRepaysDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_timestampClosed,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event
0,repay-0x851007fa5a8371105a73452ccbd7a0a4eb976e...,0x851007fa5a8371105a73452ccbd7a0a4eb976e7d2c12...,32,59865436,1676083788,0xc9e166f2737e004e93800dcef0d608b014df07f6,25,4,21,16,...,NaN,BORROWER,None,3515000787053571080985,0,0,30,21,0,Repay
1,repay-0xa243d87a35b20328eac5637a6db98058707cd0...,0xa243d87a35b20328eac5637a6db98058707cd0161df6...,61,59508912,1675978810,0xf5d0318692f822680b733955828a25507f031c95,2,0,2,1,...,1.675979e+09,BORROWER,None,0,0,0,1,1,0,Repay
2,repay-0xdee0c294dd70fb50547428f7e6d6ea6b135cb8...,0xdee0c294dd70fb50547428f7e6d6ea6b135cb8a4d75a...,33,59130546,1675866794,0x515755b2c5a209976cf0de869c30f45ac7495a60,6,0,6,12,...,1.675867e+09,BORROWER,None,0,0,0,1,1,0,Repay
3,repay-0xd6e3c80447789b02b1b9282c1624d5885a19f4...,0xd6e3c80447789b02b1b9282c1624d5885a19f40712bd...,25,59010222,1675835228,0x7ee54ab0f204bb3a83df90fdd824d8b4abe93222,3,0,3,1,...,1.675835e+09,BORROWER,None,0,0,0,1,3,0,Repay
4,repay-0xae8658b03d6aee809d73702ece0d60ced1b6be...,0xae8658b03d6aee809d73702ece0d60ced1b6be240b82...,58,58773043,1675767985,0x09fc8cd1d422e77cef172ef85cda1953915a8b74,2,0,2,1,...,1.675768e+09,BORROWER,None,0,0,0,1,1,0,Repay
5,repay-0xa02ca5cb2651d3bd78de426c9c010eea013be5...,0xa02ca5cb2651d3bd78de426c9c010eea013be53c8843...,23,58627462,1675722780,0xfa4dfbe7b012b1d92fdd5544f30518bb58bd41ce,2,0,2,1,...,1.675723e+09,BORROWER,None,0,0,0,1,2,0,Repay
6,repay-0x44899a838a0d9eaf2f48a95594a3b1041bcbdf...,0x44899a838a0d9eaf2f48a95594a3b1041bcbdfabe482...,29,58627282,1675722712,0xfa4dfbe7b012b1d92fdd5544f30518bb58bd41ce,2,0,2,1,...,1.675723e+09,BORROWER,None,0,0,0,1,2,0,Repay
7,repay-0x91f5c48839b978c11fbbe3f9502c5538060257...,0x91f5c48839b978c11fbbe3f9502c55380602570911ec...,29,58336925,1675634888,0xe2a4de267cdd4ff5ed9ba13552f5c624b12db9b2,20,6,14,65,...,NaN,BORROWER,None,78903329999999999121255,0,0,6,1,0,Repay
8,repay-0x5cb6bf86ca03bf2b59dd8265aa1e27b630ab4c...,0x5cb6bf86ca03bf2b59dd8265aa1e27b630ab4c3150a5...,32,58335805,1675634510,0x0fd84d7cb911728737556684050782b298f70f0f,7,2,5,5,...,NaN,BORROWER,None,127800000412578134238663,0,0,15,12,0,Repay
9,repay-0xdb674a81e3a0fd547a3a2ace9befd38a3b867d...,0xdb674a81e3a0fd547a3a2ace9befd38a3b867df74fc0...,30,58285118,1675618839,0x0fd84d7cb911728737556684050782b298f70f0f,7,2,5,5,...,NaN,BORROWER,None,127800000412578134238663,0,0,15,12,0,Repay


In [32]:
vestaRepaysDbTimed = db.query("select "
               "    * "
                    " ,to_timestamp(transaction_timestamp::int) dateTime "
               "From vestaRepaysDb "
               #"where lower(liquidityPools_name) like lower('%olympus%') "
                #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
               #" limit 100 "
               ).df()

vestaRepaysDbTimed.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,dateTime
0,repay-0x851007fa5a8371105a73452ccbd7a0a4eb976e...,0x851007fa5a8371105a73452ccbd7a0a4eb976e7d2c12...,32,59865436,1676083788,0xc9e166f2737e004e93800dcef0d608b014df07f6,25,4,21,16,...,BORROWER,NaN,3.515001e+21,0,0,30,21,0,Repay,2023-02-11 02:49:48
1,repay-0xa243d87a35b20328eac5637a6db98058707cd0...,0xa243d87a35b20328eac5637a6db98058707cd0161df6...,61,59508912,1675978810,0xf5d0318692f822680b733955828a25507f031c95,2,0,2,1,...,BORROWER,NaN,0.000000e+00,0,0,1,1,0,Repay,2023-02-09 21:40:10
2,repay-0xdee0c294dd70fb50547428f7e6d6ea6b135cb8...,0xdee0c294dd70fb50547428f7e6d6ea6b135cb8a4d75a...,33,59130546,1675866794,0x515755b2c5a209976cf0de869c30f45ac7495a60,6,0,6,12,...,BORROWER,NaN,0.000000e+00,0,0,1,1,0,Repay,2023-02-08 14:33:14
3,repay-0xd6e3c80447789b02b1b9282c1624d5885a19f4...,0xd6e3c80447789b02b1b9282c1624d5885a19f40712bd...,25,59010222,1675835228,0x7ee54ab0f204bb3a83df90fdd824d8b4abe93222,3,0,3,1,...,BORROWER,NaN,0.000000e+00,0,0,1,3,0,Repay,2023-02-08 05:47:08
4,repay-0xae8658b03d6aee809d73702ece0d60ced1b6be...,0xae8658b03d6aee809d73702ece0d60ced1b6be240b82...,58,58773043,1675767985,0x09fc8cd1d422e77cef172ef85cda1953915a8b74,2,0,2,1,...,BORROWER,NaN,0.000000e+00,0,0,1,1,0,Repay,2023-02-07 11:06:25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,repay-0x5aa8ad449d2938339d79f5b073fbc0404e8053...,0x5aa8ad449d2938339d79f5b073fbc0404e8053054f12...,4,46687580,1671210684,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,BORROWER,NaN,0.000000e+00,0,0,1,3,0,Repay,2022-12-16 17:11:24
96,repay-0x7957344400905e02ec95bedf686708c27e28f6...,0x7957344400905e02ec95bedf686708c27e28f6a0dbc8...,31,46686900,1671210471,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,BORROWER,NaN,0.000000e+00,0,0,1,3,0,Repay,2022-12-16 17:07:51
97,repay-0xb65bfe868838ee1b65d3313891da3118264e98...,0xb65bfe868838ee1b65d3313891da3118264e98a64277...,29,46686617,1671210388,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,BORROWER,NaN,0.000000e+00,0,0,1,3,0,Repay,2022-12-16 17:06:28
98,repay-0x7c8a50ff0c2e1e921590ae3bae1a8e9a07f76f...,0x7c8a50ff0c2e1e921590ae3bae1a8e9a07f76fc7fc93...,24,46562226,1671174745,0x68492881985afd26b04dcc4963ca5128ccd3dd9d,2,0,2,1,...,BORROWER,NaN,0.000000e+00,0,0,1,1,0,Repay,2022-12-16 07:12:25


In [33]:
vestaRepaysDbTimedLength=len(vestaRepaysDbTimed)
path =file+'/vesta_repay_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaRepaysDbTimedLength)+'_records.csv'
vestaRepaysDbTimed.to_csv(path, index = False)

path =file+'/vesta_repay_events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaRepaysDbTimedLength)+'_records_piped.csv'
vestaRepaysDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [34]:
vestaBorrowDepositEventsDb = pd.DataFrame()
vestaBorrowDepositEventsDb=pd.concat([vestaDepositsDbTimed, vestaBorrowsDbTimed])#,vestaRepaysDbTimed])
vestaBorrowDepositEventsDbLength=len(vestaBorrowDepositEventsDb)
vestaBorrowDepositEventsDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,borrows_position_blockNumberClosed,borrows_position_timestampClosed,borrows_position_side,borrows_position_isCollateral,borrows_position_balance,borrows_position_depositCount,borrows_position_withdrawCount,borrows_position_borrowCount,borrows_position_repayCount,borrows_position_liquidationCount
0,deposit-0xa8530b80a3aae63661e62d05f9c83afc5e62...,0xa8530b80a3aae63661e62d05f9c83afc5e62634b5f4c...,45.0,60064234.0,1.676143e+09,0xa13f3599ff620500f360dc6185ad30f625ebfc38,2.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,deposit-0x0712c2c572f53f272b382289d178c5801205...,0x0712c2c572f53f272b382289d178c5801205eb0b7c99...,45.0,59984711.0,1.676120e+09,0x67fa1561c5f5631de08e38d4484e0df67872f626,2.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,deposit-0xa70b833620fa9e60985ec9d9aa6b9c6af9f5...,0xa70b833620fa9e60985ec9d9aa6b9c6af9f5e2fb6ee6...,38.0,59495981.0,1.675975e+09,0x15bae977db3b62ceb5a67921fae589fd80624e8d,3.0,2.0,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,deposit-0x5d3025a16815324cf1afca689903a14d2621...,0x5d3025a16815324cf1afca689903a14d2621c19c9572...,37.0,59294798.0,1.675915e+09,0x7723c45fb37092ee65202db7a2fe562c5fe0ab48,9.0,4.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,deposit-0xabfb1cba9108afe5c2dde3f21f5b4de92fae...,0xabfb1cba9108afe5c2dde3f21f5b4de92fae79791af2...,4.0,58531581.0,1.675695e+09,0x1d883459bbc71042a9072f106427c4993de44794,2.0,2.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,deposit-0xdb674a81e3a0fd547a3a2ace9befd38a3b86...,0xdb674a81e3a0fd547a3a2ace9befd38a3b867df74fc0...,30.0,58285118.0,1.675619e+09,0x0fd84d7cb911728737556684050782b298f70f0f,7.0,2.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,deposit-0xe1f1f1cfd372bbe64a06346c1dde1102e155...,0xe1f1f1cfd372bbe64a06346c1dde1102e155be4ec282...,34.0,58200953.0,1.675596e+09,0x2682efcc31de7a06ed3bf8b0be156ff88e1663a1,7.0,2.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,deposit-0x81d0e5f18de8d3995c911b0b022d2b051866...,0x81d0e5f18de8d3995c911b0b022d2b05186668fe6c0b...,23.0,58147623.0,1.675580e+09,0x53430bde5e57ed878406a20ca101b0853f173acf,4.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,deposit-0x16bfcc7fe7939a6c9935e5062c8067a57acd...,0x16bfcc7fe7939a6c9935e5062c8067a57acd4b32abb1...,42.0,58118599.0,1.675570e+09,0xc6f84e7142eeaf930c62584f45ba7106bc3e2d68,2.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,deposit-0x61a65a717a84989f1315c17975c28846056a...,0x61a65a717a84989f1315c17975c28846056a95bb61c2...,42.0,58014267.0,1.675533e+09,0xf5d0318692f822680b733955828a25507f031c95,2.0,0.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
vestaBorrowDepositEventsDbLength = len(vestaBorrowDepositEventsDb)
path =file+'/vesta_BorrowDeposit_Events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaBorrowDepositEventsDbLength)+'_records.csv'
vestaBorrowDepositEventsDb.to_csv(path, index = False)

path =file+'/vesta_BorrowDeposit_Events_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(vestaBorrowDepositEventsDbLength)+'_records_piped.csv'
vestaBorrowDepositEventsDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [36]:
#let's pull interest rate data
interestRateTicker = 0
interestRateslist = pd.DataFrame()
interestRatesDb = pd.DataFrame()
interestRateListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (interestRateTicker)*(1000)
    interestRates = vesta.Query.interestRates(
        orderBy='id',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        #where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    interestRateslist = sg.query_df([
        interestRates.id
        ,interestRates.rate #5.21% should be stored as 5.21
        ,interestRates.duration #in days
        ,interestRates.maturityBlock #Maturity of the loan in block height. Only applies to fixed term lending (e.g. Notional)
        ,interestRates.side #The party the interest is paid to / received from
        ,interestRates.type #The type of interest rate (e.g. stable, fixed, variable, etc)
    ])

    #interestRateslist.sort_values(by=['interestRates_timestamp'])

    interestRatesDb=pd.concat([interestRateslist, interestRatesDb])
    interestRatesDbLength = len(interestRatesDb)
    interestRateListLength = len(interestRateslist)
    print("iterations: ", interestRateTicker, "Lines skipped: ",skipValue, "records collected: ", interestRateListLength)
    #print("iterations: ", interestRateTicker, "records collected: ", interestRateListLength, " - userdb length: ", interestRatesDbLength)
    interestRateTicker = interestRateTicker+1
    if interestRateListLength<1000: exit =True

print("Done. Total Records collected: ", interestRatesDbLength)




iterations:  0 Lines skipped:  0 records collected:  1000
iterations:  1 Lines skipped:  1000 records collected:  586
Done. Total Records collected:  1586


In [37]:
interestRatesDb.head(5)

,interestRates_id,interestRates_rate,interestRates_duration,interestRates_maturityBlock,interestRates_side,interestRates_type
0,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE
1,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE
2,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE
3,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE
4,BORROWER-STABLE-0x6c2c06790b3e3e3c38e12ee22f81...,0.0,None,None,BORROWER,STABLE


In [38]:
interestRatesDbLength = len(interestRatesDb)
path =file+'/vesta_interest_rates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(interestRatesDbLength)+'_records.csv'
interestRatesDb.to_csv(path, index = False)

path =file+'/vesta_interest_rates_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(interestRatesDbLength)+'_records_piped.csv'
interestRatesDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


##this WILL Capture the list of people who lended gOHM for others to borrow
lendTicker = 0
lendslist = pd.DataFrame()
lendsDb = pd.DataFrame()
lendListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (lendTicker)*(1000)
    lends = silo.Query.lends(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market

    )

    lendslist = sg.query_df([
        lends.id
        ,lends.hash
        ,lends.logIndex
        ,lends.blockNumber
        ,lends.timestamp
        ,lends.account
        ,lends.market
        ,lends.asset
        ,lends.amount
        ,lends.amountUSD
    ])

    #lendslist.sort_values(by=['lends_timestamp'])

    lendsDb=pd.concat([lendsDb, lendslist])
    recordTimestamp1 = lendsDb.iat[lendTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    lendsDbLength = len(lendsDb)
    lendListLength = len(lendslist)
    recordID = lendsDb.iat[lendTicker,0]
    lend = lendsDb.iat[lendTicker,1]
    print("iterations: ", lendTicker, "Lines skipped: ",skipValue, "records collected: ", lendListLength, " - latest recordID: ",lend, " ", recordID , " - lends DB length: ", lendsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", lendTicker, "records collected: ", lendListLength, " - userdb length: ", lendsDbLength)
    lendTicker = lendTicker+1
    if daysAgo>daysLimit: exit =True
    if lendListLength<1000 exit =True

#lendsDb.sort_values(by=['lends_timestamp'])
print("Done. Total Records collected: ", lendsDbLength)

************************************************************
FINE.... FRAX TOO

This subgraph is completely different, so we're going to  have to crunch in order to merge all these tables into coherent reports

In [39]:
frax = sg.load_api('https://gateway.thegraph.com/api/'+apiKey+'/subgraphs/id/GELTrjPJYEzxyp6Y2CtEaYpGHcJNrJA6i5Ci4KfJSEsf')

In [133]:
actionListLength=0
actionTicker = 0
fraxUsersDb = pd.DataFrame()
actionsDb = pd.DataFrame()
actionListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    #listLength=0
    skipValue = (actionTicker)*(1000)
    actionsQuery = frax.Query.actions(
         orderBy='timestamp'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.actions.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52' }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    actions= sg.query_df([
        actionsQuery.id
        ,actionsQuery.type
        ,actionsQuery.amount
        ,actionsQuery.timestamp
        ,actionsQuery.block
        ,actionsQuery.token.name
        ,actionsQuery.root.address
        ]
    )

    actions.sort_values(by=['actions_timestamp'])

    actionsDb=pd.concat([actionsDb, actions])
    recordTimestamp1 = actionsDb.iat[actionTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    actionsDbLength = len(actionsDb)
    actionListLength = len(actions)
    recordID = actionsDb.iat[actionTicker,0]
    action = actionsDb.iat[actionTicker,1]
    print("iterations: ", actionTicker, "Lines skipped: ",skipValue, "records collected: ", actionListLength, " - latest recordID: ",action, " ", recordID , " - actions DB length: ", actionsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", actionTicker, "records collected: ", actionListLength, " - userdb length: ", actionsDbLength)
    if daysAgo>daysLimit: exit =True
    if actionsDbLength<1000: exit =True
    actionTicker = actionTicker+1
print("Done. Total Records collected: ", actionsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  borrowAsset   0xa42ad3c60359e04fc73813bdabdfe157f94e370f26a21fb99077e6189e4bfd31+1  - actions DB length:  1000  - latest record from:  2023-02-12 16:56:35 0 days ago
iterations:  1 Lines skipped:  1000 records collected:  1000  - latest recordID:  addCollateral   0xa42ad3c60359e04fc73813bdabdfe157f94e370f26a21fb99077e6189e4bfd31  - actions DB length:  2000  - latest record from:  2023-02-12 16:56:35 0 days ago
iterations:  2 Lines skipped:  2000 records collected:  1000  - latest recordID:  borrowAsset   0x2f85c1a1f6bf251ca376cb2eab3fb95af43dde6c0380602d348ac6a0d1822183+1  - actions DB length:  3000  - latest record from:  2023-02-12 16:47:11 0 days ago
iterations:  3 Lines skipped:  3000 records collected:  1000  - latest recordID:  addCollateral   0x2f85c1a1f6bf251ca376cb2eab3fb95af43dde6c0380602d348ac6a0d1822183  - actions DB length:  4000  - latest record from:  2023-02-12 16:47:11 0 days ago
iterations:

In [134]:
    actionsDbTimed = db.query("select "
                   "    * "
                        " ,to_timestamp(actions_timestamp) dateTime "
                    ",(actions_amount::hugeint*1e-18) amountUsd "
                   "From actionsDb "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
actionsDbTimed.head(10)

,actions_id,actions_type,actions_amount,actions_timestamp,actions_block,actions_token_name,actions_root_address,dateTime,amountUsd
0,0xa42ad3c60359e04fc73813bdabdfe157f94e370f26a2...,borrowAsset,0.000000e+00,1676238995,16615501,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-12 21:56:35,0.000000
1,0xa42ad3c60359e04fc73813bdabdfe157f94e370f26a2...,addCollateral,9.319694e+18,1676238995,16615501,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-12 21:56:35,9.319694
2,0x2f85c1a1f6bf251ca376cb2eab3fb95af43dde6c0380...,borrowAsset,1.000000e+21,1676238431,16615454,Frax,0x054aa54e4c6f18ca76d9146d05b9c28814282fce,2023-02-12 21:47:11,1000.000000
3,0x2f85c1a1f6bf251ca376cb2eab3fb95af43dde6c0380...,addCollateral,1.000000e+18,1676238431,16615454,Governance OHM,0x054aa54e4c6f18ca76d9146d05b9c28814282fce,2023-02-12 21:47:11,1.000000
4,0xcf2e9b35e0a5c582432b78db0ac293e539f10ba35c2b...,addAsset,3.002840e+23,1676191103,16611534,Frax,0x9a67a9f0f4488a864e8d6e2c5adc4b378ab0aa28,2023-02-12 08:38:23,300284.000000
5,0xee34f32e20b41579ddf7e587f24830caee2f849ff3e7...,borrowAsset,1.000000e+22,1676181755,16610759,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-12 06:02:35,10000.000000
6,0xdbbb4f259191ec17e46adf94b08b05144a9cb953ffd4...,borrowAsset,1.500000e+22,1676180627,16610665,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-12 05:43:47,15000.000000
7,0xdbbb4f259191ec17e46adf94b08b05144a9cb953ffd4...,addCollateral,1.868763e+19,1676180627,16610665,Governance OHM,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-12 05:43:47,18.687634
8,0x06a7261f4c8dcf2898b2041e79f8ee5f3f59a3fc7b4b...,borrowAsset,5.000000e+21,1676121263,16605746,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-11 13:14:23,5000.000000
9,0x40aae84af13261a43d09f402c7cfb666080135218ece...,borrowAsset,1.000000e+22,1676120543,16605686,Frax,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-11 13:02:23,10000.000000


In [135]:
actionsDbTimedLength = len(actionsDbTimed)
path =file+'/frax_actions_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(actionsDbTimedLength)+'_records.csv'
actionsDbTimed.to_csv(path, index = False)

path =file+'/frax_actions_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(actionsDbTimedLength)+'_records_piped.csv'
actionsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [140]:
fraxBorrowsDb = db.query("select distinct"
                   "    actions_root_address "
                         ", dateTime "
                         ", amountUSD "
                         ", actions_token_name "
                       "From actionsDbTimed "
                       "where actions_type= 'borrowAsset' "
                   ).df()
fraxBorrowsDb.head(10)

,actions_root_address,dateTime,amountUsd,actions_token_name
0,0xf604556e7d829240424d51d2c5f2c1861adb3a26,2023-01-08 00:34:47,1000.0,Frax
1,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,2023-01-07 22:07:23,7000.0,Frax
2,0x0430c0cd447397d5411aea19933165bd96c46fe3,2023-01-07 04:20:47,64000.0,Frax
3,0xc0e0de939f26e71683452467942aff455ca153e3,2023-01-06 15:31:47,0.0,Frax
4,0xc0e0de939f26e71683452467942aff455ca153e3,2023-01-06 15:28:23,2500.0,Frax
5,0xc0e0de939f26e71683452467942aff455ca153e3,2023-01-06 13:29:59,0.0,Frax
6,0xc0e0de939f26e71683452467942aff455ca153e3,2023-01-06 13:27:35,5000.0,Frax
7,0x440c8ab91ba7c8763b3d9b3357dc1c0d13a148aa,2023-01-05 18:21:23,700.0,Frax
8,0x4731223a7b5ac0c425f1f35d0bdf6d340a93a181,2023-01-05 11:17:11,500.0,Frax
9,0x440c8ab91ba7c8763b3d9b3357dc1c0d13a148aa,2023-01-04 18:54:35,1000.0,Frax


**********************************************************
LET'S PLAY AND CRUNCH DATA - BORROWS


In [141]:
fraxBorrowsDb.drop_duplicates

fraxBorrowsDbLength = len(fraxBorrowsDb)
path =file+'/frax_borrows_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(fraxBorrowsDbLength)+'_records.csv'
fraxBorrowsDb.to_csv(path, index = False)

path =file+'/frax_borrows_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(fraxBorrowsDbLength)+'_records_piped.csv'
fraxBorrowsDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [144]:
fraxBorrowedUSDDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(amountUsd) amountUsdBorrowed "
                        "   ,count(distinct actions_root_address) borrowers "
                       "From fraxBorrowsDb "
                        "Group by 1 "
                        "order by 1"
                   ).df()
fraxBorrowedUSDDaily['protocol'] = 'Frax'
fraxBorrowedUSDDaily.head(10)

,calDay,amountUsdBorrowed,borrowers,protocol
0,2022-12-03,64968.786050,8,Frax
1,2022-12-04,345329.334643,7,Frax
2,2022-12-05,508933.348518,5,Frax
3,2022-12-06,5000.000000,1,Frax
4,2022-12-07,206250.000000,3,Frax
5,2022-12-08,531726.823715,7,Frax
6,2022-12-09,288250.000000,4,Frax
7,2022-12-10,160000.000000,2,Frax
8,2022-12-11,8000.000000,1,Frax
9,2022-12-12,14125.000000,4,Frax


fraxCollateralDb = db.query("select "
                   "    * "
                       "From actionsDbTimed "
                       "where actions_type= 'addCollateral' "
                   ).df()

fraxCollateralDb.head(10)

In [145]:
siloBorrowedUSDDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(transaction_amountUSD::hugeint) amountUsdBorrowed "
                        "   ,count(distinct transaction_account_id) borrowers "
                       "From borrowsDbTimed "
                        "Group by 1 "
                   ).df()
siloBorrowedUSDDaily['protocol'] = 'Silo'
siloBorrowedUSDDaily.head(10)

,calDay,amountUsdBorrowed,borrowers,protocol
0,2023-02-09,1122006.0,2,Silo
1,2023-02-08,2382.0,1,Silo
2,2023-02-07,526416.0,5,Silo
3,2023-02-01,30006.0,1,Silo
4,2023-01-28,60246.0,1,Silo
5,2022-12-17,2988.0,1,Silo
6,2022-12-13,48246.0,1,Silo
7,2022-12-12,95646.0,2,Silo
8,2022-11-23,3966.0,2,Silo


In [146]:
vestaBorrowedUSDDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(borrows_amountUSD::hugeint) amountUsdBorrowed "
                        "   ,count(distinct borrows_account_id) borrowers "
                       "From vestaBorrowsDbTimed "
                        "Group by 1 "
                   ).df()
vestaBorrowedUSDDaily['protocol'] = 'Vesta'
vestaBorrowedUSDDaily.head(10)

,calDay,amountUsdBorrowed,borrowers,protocol
0,2023-02-12,91777.0,1,Vesta
1,2023-02-11,7615.0,2,Vesta
2,2023-02-10,261.0,1,Vesta
3,2023-02-09,50100.0,2,Vesta
4,2023-02-08,25881.0,1,Vesta
5,2023-02-07,126480.0,3,Vesta
6,2023-02-06,14572.0,1,Vesta
7,2023-02-05,7583.0,3,Vesta
8,2023-02-04,2286.0,1,Vesta
9,2023-02-03,54463.0,2,Vesta


In [147]:
#Don't judge if I use pandas for some things and SQL for others. I'm a noob :P
dailyUsdBorrowdDb =  pd.concat([vestaBorrowedUSDDaily, siloBorrowedUSDDaily,fraxBorrowedUSDDaily])
dailyUsdBorrowdDb.head(400)

,calDay,amountUsdBorrowed,borrowers,protocol
0,2023-02-12,91777.0,1,Vesta
1,2023-02-11,7615.0,2,Vesta
2,2023-02-10,261.0,1,Vesta
3,2023-02-09,50100.0,2,Vesta
4,2023-02-08,25881.0,1,Vesta
...,...,...,...,...
64,2023-02-08,95000.0,2,Frax
65,2023-02-09,187600.0,5,Frax
66,2023-02-10,25000.0,1,Frax
67,2023-02-11,47500.0,2,Frax


In [148]:
dailyUsdBorrowdDbLength = len(dailyUsdBorrowdDb)
path =file+'/dailyUsdBorrowed_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(dailyUsdBorrowdDbLength)+'_records.csv'
dailyUsdBorrowdDb.to_csv(path, index = False)

path =file+'/dailyUsdBorrowed_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(dailyUsdBorrowdDbLength)+'_records_piped.csv'
dailyUsdBorrowdDb.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [48]:
dailyUsdBorrowdDbLength

388

*******************************8
REPAYS!

In [49]:
fraxRepaysDb = db.query("select "
                   "    * "
                    ",(actions_amount::hugeint*1e-18) amountUsdRepayed "
                       "From actionsDbTimed "
                       "where actions_type= 'repayAsset' "
                   ).df()
fraxRepaysDb.head(10)

,actions_id,actions_type,actions_amount,actions_share,actions_timestamp,actions_block,actions_token_id,actions_token_address,actions_token_decimals,actions_token_name,actions_token_symbol,actions_token_totalSupply,actions_token_pairAssetCount,actions_token_pairCollateralCount,actions_root_id,actions_root_address,dateTime,amountUsdRepayed
0,0x61b85113554f41e41beb5a81ae45336168b6eedaf7c5...,repayAsset,1.300000e+21,1.280695e+21,1676079827,16602316,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0xf10918fc1725a65960db2b906b4868a0fdbb6870,0xf10918fc1725a65960db2b906b4868a0fdbb6870,2023-02-11 01:43:47,1300.000128
1,0xc7c1739911ae71f7726d1d32214edb8161a341f7fcd7...,repayAsset,8.128943e+20,8.008463e+20,1676068895,16601407,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0x46535cbc3fa574c32ea392d4f89c518290597072,0x46535cbc3fa574c32ea392d4f89c518290597072,2023-02-10 22:41:35,812.894298
2,0x40e939c94b527bd4eb9f8f102cfe3c179b61f0da6bb7...,repayAsset,2.000000e+22,1.970463e+22,1676049335,16599784,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0xc0e0de939f26e71683452467942aff455ca153e3,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-10 17:15:35,20000.000658
3,0x83e2ea08ec4f8a38735c7ad01954ce62ae9f79b84e7d...,repayAsset,6.625100e+22,6.527284e+22,1676047871,16599664,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0xc0e0de939f26e71683452467942aff455ca153e3,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-10 16:51:11,66251.004408
4,0x5dee647c2e8ebfad8bac45dd1ca3eab67b4841ef1692...,repayAsset,7.983305e+22,7.865760e+22,1676032955,16598429,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0x46535cbc3fa574c32ea392d4f89c518290597072,0x46535cbc3fa574c32ea392d4f89c518290597072,2023-02-10 12:42:35,79833.054078
5,0x7f9a9b4079f340cc010ddcf1ccbac3db2e1ea4e29adb...,repayAsset,7.999901e+22,7.884488e+22,1675924787,16589468,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0x46535cbc3fa574c32ea392d4f89c518290597072,0x46535cbc3fa574c32ea392d4f89c518290597072,2023-02-09 06:39:47,79999.010768
6,0xbc537609adb35fdd165f22a83da392b898080e7e2b70...,repayAsset,7.999901e+22,7.885011e+22,1675900835,16587479,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0x46535cbc3fa574c32ea392d4f89c518290597072,0x46535cbc3fa574c32ea392d4f89c518290597072,2023-02-09 00:00:35,79999.005384
7,0x87953dce0f55cdd157c52fb9ce2ede8c34012f0c2c29...,repayAsset,2.522200e+22,2.486499e+22,1675824611,16581186,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0xc0e0de939f26e71683452467942aff455ca153e3,0xc0e0de939f26e71683452467942aff455ca153e3,2023-02-08 02:50:11,25222.001669
8,0x780b2ae8e35f8c6155032b8211eb8e1fbdb0011af914...,repayAsset,6.051001e+21,5.966598e+21,1675748243,16574865,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,0x6487d336bdec94d4714a4d217b15aa5c7b9dfa99,2023-02-07 05:37:23,6051.000596
9,0xea0b3054fc21bd86b06f4f015561d14f12fa49c1e112...,repayAsset,7.438101e+22,7.336350e+22,1675643123,16566149,0x853d955acef822db058eb8505911ed77f175b99e,0x853d955acef822db058eb8505911ed77f175b99e,18,Frax,FRAX,1.491128e+27,12,0,0x46535cbc3fa574c32ea392d4f89c518290597072,0x46535cbc3fa574c32ea392d4f89c518290597072,2023-02-06 00:25:23,74381.006973


In [50]:
fraxRepaiddUsdDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(amountUsdRepayed) amountUsdRepaid "
                        "   ,count(distinct actions_root_id) repayers "
                       "From fraxRepaysDb "
                        "Group by 1 "
                        "order by 1"
                   ).df()
fraxRepaiddUsdDaily['protocol'] = 'Frax'
fraxRepaiddUsdDaily.head(10)

,calDay,amountUsdRepaid,repayers,protocol
0,2022-12-03,1.491082e+06,2,Frax
1,2022-12-04,3.075388e+07,3,Frax
2,2022-12-05,4.921373e+06,1,Frax
3,2022-12-06,9.940000e+04,1,Frax
4,2022-12-07,7.581233e+07,1,Frax
5,2022-12-08,3.978840e+08,2,Frax
6,2022-12-09,9.939941e+07,1,Frax
7,2022-12-10,3.482573e+07,1,Frax
8,2022-12-11,1.149717e+06,1,Frax
9,2022-12-12,1.819163e+07,2,Frax


In [51]:
vestaUsdRepaidDaily = db.query(
                        "select "
                            "date_trunc('day',dateTime) calDay "
                            ",sum(transaction_amountUSD::hugeint) amountUsdRepaid "
                        "   ,count(distinct transaction_account_id) repayers "
                       "From  vestaRepaysDbTimed "
                        "Group by 1 "
                   ).df()
vestaUsdRepaidDaily['protocol'] = 'Vesta'
vestaUsdRepaidDaily.head(10)

,calDay,amountUsdRepaid,repayers,protocol
0,2023-02-11,150.0,1,Vesta
1,2023-02-09,2286.0,1,Vesta
2,2023-02-08,31182.0,2,Vesta
3,2023-02-07,13173.0,1,Vesta
4,2023-02-06,1000.0,1,Vesta
5,2023-02-05,124831.0,2,Vesta
6,2023-02-02,10768.0,3,Vesta
7,2023-02-01,20270.0,2,Vesta
8,2023-01-31,1586.0,1,Vesta
9,2023-01-25,882.0,1,Vesta


In [52]:
SiloUsdRepaidDaily = db.query(
                        "select "
                            "date_trunc('day',to_timestamp(transaction_timestamp::int)) calDay "
                            ",sum(transaction_amountUSD::hugeint) amountUsdRepaid "
                        "   ,count(distinct transaction_account_id) repayers "
                       "From  repaysDb "
                        "Group by 1 "
                   ).df()
SiloUsdRepaidDaily['protocol'] = 'Silo'
SiloUsdRepaidDaily.head(10)

,calDay,amountUsdRepaid,repayers,protocol
0,2022-11-25,2802.0,1,Silo
1,2022-11-24,1572.0,1,Silo


*****************************
DAILY SNAPSHOTS - FRAX FIRST

In [53]:
pairSnapshotsListLength=0
pairSnapshotsTicker = 0
Snapshots = pd.DataFrame()
pairSnapshotsDb = pd.DataFrame()
pairSnapshotsListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit= False
while exit==False:
    #listLength=0
    skipValue = (pairSnapshotsTicker)*(1000)
    pairsQuery = frax.Query.pairPerDays(
         orderBy='id'
        ,orderDirection='desc'
        ,first=1000
        ,skipValue = skipValue
        #,where = [frax.Query.pairs.token.id == '0x66bf36dba79d4606039f04b32946a260bcd3ff52']
        ,where={ 'pair': '0x66bf36dba79d4606039f04b32946a260bcd3ff52' }
        #,where = {'positions.pairid':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}
    #{'users_positions_pair_id':'0x66bf36dba79d4606039f04b32946a260bcd3ff52'}

    )
    Snapshots= sg.query_df([
    pairsQuery.id
    ,pairsQuery.feeToProtocolRate
    ,pairsQuery.exchangeRate
    ,pairsQuery.totalAssetAmount
    ,pairsQuery.totalAssetShare
    ,pairsQuery.totalCollateral
    ,pairsQuery.totalBorrowAmount
    ,pairsQuery.totalBorrowShare
    ,pairsQuery.interestPerSecond
    ,pairsQuery.utilization
    ,pairsQuery.totalFeesAmount
    ,pairsQuery.totalFeesShare
    ,pairsQuery.lastAccrued
    ,pairsQuery.block
    ,pairsQuery.timestamp
    ,pairsQuery.totalAssetValue
    ,pairsQuery.totalCollateralValue
    ,pairsQuery.totalBorrowValue
    ,pairsQuery.pair
    ]
    )

    Snapshots.sort_values(by=['pairPerDays_timestamp'])

    pairSnapshotsDb=pd.concat([pairSnapshotsDb, Snapshots])
    recordTimestamp1 = pairSnapshotsDb.iat[pairSnapshotsTicker,14]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    pairSnapshotsDbLength = len(pairSnapshotsDb)
    pairSnapshotsListLength = len(Snapshots)
    recordID = pairSnapshotsDb.iat[pairSnapshotsTicker,0]
    Snapshot = pairSnapshotsDb.iat[pairSnapshotsTicker,1]
    print("iterations: ", pairSnapshotsTicker, "Lines skipped: ",skipValue, "records collected: ", pairSnapshotsListLength, " - latest recordID: ",Snapshot, " ", recordID , " - pairSnapshots DB length: ", pairSnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", pairSnapshotsTicker, "records collected: ", pairSnapshotsListLength, " - userdb length: ", pairSnapshotsDbLength)
    if daysAgo>daysLimit: exit =True
    if pairSnapshotsListLength<1000: exit =True
    pairSnapshotsTicker = pairSnapshotsTicker+1

print("Done. Total Records collected: ", pairSnapshotsDbLength)

iterations:  0 Lines skipped:  0 records collected:  77  - latest recordID:  0   0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent  - pairSnapshots DB length:  77  - latest record from:  2023-02-12 16:56:35 0 days ago
Done. Total Records collected:  77


In [54]:
fraxPairSnapshotsDbTimed= db.query("select "
                   "    * "
                        " ,to_timestamp(pairPerDays_timestamp) dateTime "
                   "From pairSnapshotsDb "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
fraxPairSnapshotsDbTimed.head(1000)

,pairPerDays_id,pairPerDays_feeToProtocolRate,pairPerDays_exchangeRate,pairPerDays_totalAssetAmount,pairPerDays_totalAssetShare,pairPerDays_totalCollateral,pairPerDays_totalBorrowAmount,pairPerDays_totalBorrowShare,pairPerDays_interestPerSecond,pairPerDays_utilization,...,pairPerDays_pair_oracleMultiplyAddress_id,pairPerDays_pair_oracleDivideAddress_id,pairPerDays_pair_rateContract_id,pairPerDays_pair_maxLTV,pairPerDays_pair_liquidationFee,pairPerDays_pair_maturity,pairPerDays_pair_pauseStatus,pairPerDays_pair_lenderWhitelistActive,pairPerDays_pair_borrowerWhitelistActive,dateTime
0,0x66bf36dba79d4606039f04b32946a260bcd3ff52-Recent,0,378416746559582,5.213638e+24,5.149945e+24,2.924565e+21,3.885433e+24,3.825964e+24,2625170298,74524,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-12 21:56:35
1,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19401,0,378416746559582,5.213638e+24,5.149945e+24,2.924565e+21,3.885433e+24,3.825964e+24,2625170298,74524,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-12 21:56:35
2,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19400,0,378555412205106,5.213593e+24,5.149945e+24,2.924565e+21,3.885388e+24,3.825964e+24,2625172870,74524,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-12 21:56:35
3,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19399,0,371588713173683,4.912413e+24,4.853295e+24,2.895558e+21,3.858491e+24,3.800358e+24,2758282929,78545,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-11 13:14:23
4,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19398,0,377188472125277,4.911010e+24,4.852807e+24,2.891463e+21,3.811382e+24,3.754848e+24,2727331303,77608,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2023-02-10 22:42:47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19330,0,416982985435782,3.000266e+24,2.999682e+24,1.702268e+21,1.642246e+24,1.641700e+24,887806114,54736,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2022-12-04 23:12:59
73,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19329,0,435238970363534,3.000407e+24,2.999932e+24,1.460558e+21,1.358687e+24,1.358333e+24,797470641,45283,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2022-12-03 23:30:59
74,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19328,0,424429606966213,3.000316e+24,2.999932e+24,1.298896e+21,1.287539e+24,1.287292e+24,819794982,42913,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2022-12-02 23:55:59
75,0x66bf36dba79d4606039f04b32946a260bcd3ff52-19327,0,455232217741157,1.000250e+24,1.000178e+24,1.165443e+17,1.821276e+20,1.821093e+20,158550393,18,...,0x0000000000000000000000000000000000000000,0xe893297a9d4310976424fd0b25f53ac2b6464fe3,0x18500cb1f2fe7a40ebda393383a0b8548a31f261,75000,10000,0,False,False,False,2022-12-01 21:30:47


*************************************************************************
SILO - ah turns out we haven't launched yet as of 02/11

In [56]:
siloInterestDailySnapshotTicker = 0
siloInterestDailySnapshotslist = pd.DataFrame()
siloInterestDailySnapshotsDb = pd.DataFrame()
siloInterestDailySnapshotListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (siloInterestDailySnapshotTicker)*(1000)
    siloInterestDailySnapshots = silo.Query.interestRateDailies(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip= skipValue,
        #where={ 'market': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0' } #gOHM market
        #where = {'day': 19400, 'timestamp':1676160000}
        where = {'interestRate_in': ['LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac','LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2','LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5','LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac','LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2','LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0x0ab87046fbb341d058f17cbc4c1133f25a20a52f','BORROWER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac','BORROWER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2','BORROWER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5','BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac','BORROWER-VARIABLE-0xc413dd03555f3eb29d834b482d386b2999dc2eb0-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']}
    )

    siloInterestDailySnapshotslist = sg.query_df([
        siloInterestDailySnapshots.interestRate.market.name
        ,siloInterestDailySnapshots.id
        ,siloInterestDailySnapshots.day
        ,siloInterestDailySnapshots.timestamp
        ,siloInterestDailySnapshots.rateLast #The most recent rate recorded or last for the day
        ,siloInterestDailySnapshots.rateHigh #Highest the rate was for the day
        ,siloInterestDailySnapshots.rateLow
        ,siloInterestDailySnapshots.rateAvg  #Interest rate in percentage APY. E.g. 5.21% should be stored as 5.21
        ,siloInterestDailySnapshots.interestRate.side
        ,siloInterestDailySnapshots.interestRate.type
    ])

    #siloInterestDailySnapshotslist.sort_values(by=['siloInterestDailySnapshots_timestamp'])

    siloInterestDailySnapshotsDb=pd.concat([siloInterestDailySnapshotsDb, siloInterestDailySnapshotslist])
    recordTimestamp1 = siloInterestDailySnapshotsDb.iat[siloInterestDailySnapshotTicker,3]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    siloInterestDailySnapshotsDbLength = len(siloInterestDailySnapshotsDb)
    siloInterestDailySnapshotListLength = len(siloInterestDailySnapshotslist)
    recordID = siloInterestDailySnapshotsDb.iat[siloInterestDailySnapshotTicker,0]
    siloInterestDailySnapshot = siloInterestDailySnapshotsDb.iat[siloInterestDailySnapshotTicker,1]
    print("iterations: ", siloInterestDailySnapshotTicker, "Lines skipped: ",skipValue, "records collected: ", siloInterestDailySnapshotListLength, " - latest recordID: ",siloInterestDailySnapshot, " ", recordID , " - siloInterestDailySnapshots DB length: ", siloInterestDailySnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", siloInterestDailySnapshotTicker, "records collected: ", siloInterestDailySnapshotListLength, " - userdb length: ", siloInterestDailySnapshotsDbLength)
    siloInterestDailySnapshotTicker = siloInterestDailySnapshotTicker+1

    if daysAgo>daysLimit: exit =True
    if siloInterestDailySnapshotListLength<1000: exit =True
print("Done. Total Records collected: ", siloInterestDailySnapshotsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xd7c9f0e536dc865ae858b0c0453fe76d13c3beac-19401   OHM  - siloInterestDailySnapshots DB length:  1000  - latest record from:  2023-02-12 19:00:00 0 days ago
iterations:  1 Lines skipped:  1000 records collected:  107  - latest recordID:  LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e0820f68657d7db-0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2-19401   OHM  - siloInterestDailySnapshots DB length:  1107  - latest record from:  2023-02-12 19:00:00 0 days ago
Done. Total Records collected:  1107


In [57]:
print(len(siloInterestDailySnapshotsDb))
siloInterestDailySnapshotsDb.drop_duplicates
print(len(siloInterestDailySnapshotsDb))

1107
1107


In [58]:
siloInterestDailySnapshotsDbTimed= db.query("select "
                   "    * "
                        " ,to_timestamp(interestRateDailies_timestamp) dateTime "
                   "From siloInterestDailySnapshotsDb "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
siloInterestDailySnapshotsDbTimed.head(1000)

,interestRateDailies_interestRate_market_name,interestRateDailies_id,interestRateDailies_day,interestRateDailies_timestamp,interestRateDailies_rateLast,interestRateDailies_rateHigh,interestRateDailies_rateLow,interestRateDailies_rateAvg,interestRateDailies_interestRate_side,interestRateDailies_interestRate_type,dateTime
0,OHM,LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e08...,19401,1676246400,0.000000,0.000000,0.000000,0.000000,LENDER,VARIABLE,2023-02-13
1,OHM,LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e08...,19401,1676246400,0.000000,0.000000,0.000000,0.000000,LENDER,VARIABLE,2023-02-13
2,OHM,LENDER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e08...,19401,1676246400,0.000000,0.000000,0.000000,0.000000,LENDER,VARIABLE,2023-02-13
3,gOHM,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,19401,1676246400,2.148783,2.148783,2.148783,2.148783,LENDER,VARIABLE,2023-02-13
4,gOHM,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,19401,1676246400,0.000000,0.000000,0.000000,0.000000,LENDER,VARIABLE,2023-02-13
...,...,...,...,...,...,...,...,...,...,...,...
995,gOHM,LENDER-VARIABLE-0xc413dd03555f3eb29d834b482d38...,19310,1668384000,0.000000,0.000000,0.000000,0.000000,LENDER,VARIABLE,2022-11-14
996,OHM,BORROWER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e...,19310,1668384000,0.000000,0.000000,0.000000,0.000000,BORROWER,VARIABLE,2022-11-14
997,OHM,BORROWER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e...,19310,1668384000,0.000000,0.000000,0.000000,0.000000,BORROWER,VARIABLE,2022-11-14
998,OHM,BORROWER-VARIABLE-0xf5ffabab8f9a6f4f6de1f0dd6e...,19310,1668384000,0.000000,0.000000,0.000000,0.000000,BORROWER,VARIABLE,2022-11-14


In [59]:
siloInterestDailySnapshotsDbTimedLength = len(siloInterestDailySnapshotsDbTimed)
path =file+'/siloInterestDailySnapshotsDbTimedLength_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloInterestDailySnapshotsDbTimedLength)+'_records.csv'
siloInterestDailySnapshotsDbTimed.to_csv(path, index = False)

path =file+'/siloInterestDailySnapshotsDbTimedLength_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(siloInterestDailySnapshotsDbTimedLength)+'_records_piped.csv'
siloInterestDailySnapshotsDbTimed.to_csv(path, index = False, sep='|')
print('file saved')

file saved


In [72]:
siloDailySnapshot = silo.Query.marketDailySnapshots(
    orderBy='timestamp',
    orderDirection='desc',
    first=10,
    #skip = skipValue,
    where={              'market':'0xc413dd03555f3eb29d834b482d386b2999dc2eb0'            } #gOHM market
    #where={              'market':'0xfccc27aabd0ab7a0b2ad2b7760037b1eab61616b'            } #DAI for testing
    #0x0ab87046fbb341d058f17cbc4c1133f25a20a52f
)

siloDailySnapshotslist = sg.query_df([
    siloDailySnapshot.protocol.name
    ,siloDailySnapshot.protocol.type
    ,siloDailySnapshot.protocol.cumulativeUniqueUsers
    ,siloDailySnapshot.protocol.cumulativeUniqueDepositors
    ,siloDailySnapshot.protocol.cumulativeUniqueBorrowers
    ,siloDailySnapshot.protocol.cumulativeUniqueLiquidators
    ,siloDailySnapshot.protocol.cumulativeUniqueLiquidatees
    ,siloDailySnapshot.protocol.totalValueLockedUSD
    ,siloDailySnapshot.protocol.protocolControlledValueUSD
    ,siloDailySnapshot.protocol.cumulativeSupplySideRevenueUSD
    ,siloDailySnapshot.protocol.cumulativeProtocolSideRevenueUSD
    ,siloDailySnapshot.protocol.cumulativeTotalRevenueUSD
    ,siloDailySnapshot.protocol.totalDepositBalanceUSD
    ,siloDailySnapshot.protocol.cumulativeDepositUSD
    ,siloDailySnapshot.protocol.cumulativeWithdrawUSD
    ,siloDailySnapshot.protocol.totalBorrowBalanceUSD
    ,siloDailySnapshot.protocol.totalBorrowBalanceUSD
    ,siloDailySnapshot.protocol.cumulativeLiquidateUSD
    ,siloDailySnapshot.protocol.creditSupply
    ,siloDailySnapshot.protocol.creditUtilized
    ,siloDailySnapshot.market.id
    ,siloDailySnapshot.market.name
    ,siloDailySnapshot.market.totalValueLockedUSD
    #,siloDailySnapshot.market.totalControlledValueUSD
    ,siloDailySnapshot.market.cumulativeSupplySideRevenueUSD
    ,siloDailySnapshot.market.cumulativeTotalRevenueUSD
    ,siloDailySnapshot.market.cumulativeSupplySideRevenueUSD
    ,siloDailySnapshot.market.cumulativeProtocolSideRevenueUSD
    ,siloDailySnapshot.market.totalDepositBalanceUSD
    ,siloDailySnapshot.market.cumulativeDepositUSD
    ,siloDailySnapshot.market.totalBorrowBalanceUSD
    ,siloDailySnapshot.market.cumulativeBorrowUSD
    ,siloDailySnapshot.market.cumulativeLiquidateUSD
    ,siloDailySnapshot.market.creditSupply
    ,siloDailySnapshot.market.creditUtilized
    ,siloDailySnapshot.timestamp
])

siloDailySnapshotslistTimed= db.query("select "
                   "    * "
                        " ,to_timestamp(marketDailySnapshots_timestamp) dateTime "
                   "From siloDailySnapshotslist "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
siloDailySnapshotslistTimed.drop_duplicates
siloDailySnapshotslistTimed.head(1000)

,marketDailySnapshots_protocol_name,marketDailySnapshots_protocol_type,marketDailySnapshots_protocol_cumulativeUniqueUsers,marketDailySnapshots_protocol_cumulativeUniqueDepositors,marketDailySnapshots_protocol_cumulativeUniqueBorrowers,marketDailySnapshots_protocol_cumulativeUniqueLiquidators,marketDailySnapshots_protocol_cumulativeUniqueLiquidatees,marketDailySnapshots_protocol_totalValueLockedUSD,marketDailySnapshots_protocol_protocolControlledValueUSD,marketDailySnapshots_protocol_cumulativeSupplySideRevenueUSD,...,marketDailySnapshots_market_cumulativeProtocolSideRevenueUSD,marketDailySnapshots_market_totalDepositBalanceUSD,marketDailySnapshots_market_cumulativeDepositUSD,marketDailySnapshots_market_totalBorrowBalanceUSD,marketDailySnapshots_market_cumulativeBorrowUSD,marketDailySnapshots_market_cumulativeLiquidateUSD,marketDailySnapshots_market_creditSupply,marketDailySnapshots_market_creditUtilized,marketDailySnapshots_timestamp,dateTime
0,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1676072123,2023-02-10 23:35:23
1,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1675964891,2023-02-09 17:48:11
2,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1675884863,2023-02-08 19:34:23
3,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1675811711,2023-02-07 23:15:11
4,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1675694339,2023-02-06 14:38:59
5,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1675273067,2023-02-01 17:37:47
6,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1674908519,2023-01-28 12:21:59
7,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1674641375,2023-01-25 10:09:35
8,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1673897411,2023-01-16 19:30:11
9,Silo Finance,LENDING,187,180,111,14,8,2.724117e+07,2.724117e+07,38258.335674,...,0.0,707845.598606,1.240736e+06,317206.9413,315317.889843,0.0,4.000000e+23,3.149118e+23,1673230019,2023-01-09 02:06:59


In [61]:
siloDailySnapshot = silo.Query.marketDailySnapshots(
    orderBy='timestamp',
    orderDirection='desc',
    #first=1000,
    #skip = skipValue,
    where={ 'id': '0xc413dd03555f3eb29d834b482d386b2999dc2eb0-19398'
            ,'protocol':'0xd998c35b7900b344bbbe6555cc11576942cf309d'
            #,'market':'gOHM'
            } #gOHM market
    #0x0ab87046fbb341d058f17cbc4c1133f25a20a52f
)

siloDailySnapshotslist = sg.query_df([
    siloDailySnapshot.id
    ,siloDailySnapshot.protocol
    ,siloDailySnapshot.market
    ,siloDailySnapshot.timestamp
])
siloDailySnapshotslist.head(10)

,marketDailySnapshots_id,marketDailySnapshots_protocol_id,marketDailySnapshots_protocol_name,marketDailySnapshots_protocol_slug,marketDailySnapshots_protocol_schemaVersion,marketDailySnapshots_protocol_subgraphVersion,marketDailySnapshots_protocol_methodologyVersion,marketDailySnapshots_protocol_network,marketDailySnapshots_protocol_type,marketDailySnapshots_protocol_lendingType,...,marketDailySnapshots_market_createdTimestamp,marketDailySnapshots_market_createdBlockNumber,marketDailySnapshots_market_positionCount,marketDailySnapshots_market_openPositionCount,marketDailySnapshots_market_closedPositionCount,marketDailySnapshots_market_lendingPositionCount,marketDailySnapshots_market_borrowingPositionCount,marketDailySnapshots_market_creditSupply,marketDailySnapshots_market_creditUtilized,marketDailySnapshots_timestamp
0,0xc413dd03555f3eb29d834b482d386b2999dc2eb0-19398,0xd998c35b7900b344bbbe6555cc11576942cf309d,Silo Finance,silo-finance,2.0.1,2.0.0,1.0.0,MAINNET,LENDING,POOLED,...,1667314979,15875945,46,34,12,31,15,400000000000000000000000,314911774201888078106727,1676072123


*****************************************************
THIS... IS.... VESTAAAAAAAAAAA!!
(kick)

In [62]:
##this captures the ENTIRE list of people who vestaDailySnapshoted XAI against gOHM
vestaDailySnapshotTicker = 0
vestaDailySnapshotslist = pd.DataFrame()
vestaDailySnapshotsDb = pd.DataFrame()
vestaDailySnapshotListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaDailySnapshotTicker)*(1000)
    vestaDailySnapshots = vesta.Query.MarketDailySnapshot(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaDailySnapshotslist = sg.query_df([
        vestaDailySnapshots.id
        ,vestaDailySnapshots.market
        ,vestaDailySnapshots.rates
    ])

    #vestaDailySnapshotslist.sort_values(by=['vestaDailySnapshots_timestamp'])

    vestaDailySnapshotsDb=pd.concat([vestaDailySnapshotsDb, vestaDailySnapshotslist])
    recordTimestamp1 = vestaDailySnapshotsDb.iat[vestaDailySnapshotTicker,2]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaDailySnapshotsDbLength = len(vestaDailySnapshotsDb)
    vestaDailySnapshotListLength = len(vestaDailySnapshotslist)
    recordID = vestaDailySnapshotsDb.iat[vestaDailySnapshotTicker,0]
    vestaDailySnapshot = vestaDailySnapshotsDb.iat[vestaDailySnapshotTicker,1]
    print("iterations: ", vestaDailySnapshotTicker, "Lines skipped: ",skipValue, "records collected: ", vestaDailySnapshotListLength, " - latest recordID: ",vestaDailySnapshot, " ", recordID , " - vestaDailySnapshots DB length: ", vestaDailySnapshotsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaDailySnapshotTicker, "records collected: ", vestaDailySnapshotListLength, " - userdb length: ", vestaDailySnapshotsDbLength)
    vestaDailySnapshotTicker = vestaDailySnapshotTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaDailySnapshotListLength<1000: exit =True


vestaDailySnapshotsDb.columns = vestaDailySnapshotsDb.columns.str.replace("vestaDailySnapshots", "transaction")
vestaDailySnapshotsDb['event']='vestaDailySnapshot'
print("Done. Total Records collected: ", vestaDailySnapshotsDbLength)

KeyError: 'TypeMeta.ObjectMeta.field: no field named MarketDailySnapshot for interface Query'

In [ ]:
vestaDailySnapshots = vesta.Query.marketDailySnapshots(
    orderBy='timestamp',
    orderDirection='desc',
    first=1000,
    #where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market
    #WHERE = {'interestRateDailies.interestRate.market.name': 'XAI'}
    #where = {'marketDailySnapshots.market.name': 'Olympus Governance Token'}
)

vestaDailySnapshotslist = sg.query_df([
    vestaDailySnapshots.id
    ,vestaDailySnapshots.timestamp
    ,vestaDailySnapshots.market
    ,vestaDailySnapshots.rates
    ,vestaDailySnapshots.totalValueLockedUSD
    ,vestaDailySnapshots.dailyBorrowUSD
    ,vestaDailySnapshots.dailyDepositUSD
    ,vestaDailySnapshots.dailyBorrowUSD
    ,vestaDailySnapshots.dailyProtocolSideRevenueUSD
    ,vestaDailySnapshots.cumulativeProtocolSideRevenueUSD




],
#pagination_strategy=ShallowStrategy
)
vestaDailySnapshotslist.head(1000)

In [71]:
##Loan LEngth
fraxLoanLengthDB = db.query("select distinct "
                        "    A.actions_root_id "
                            ",sum(A.actions_amountUSD) totalBorrowed "
                            ",sum(B.actions_amountUSD) totalRepaid "
                            " ,sum(A.actions_amountUSD) - sum(B.actions_amountUSD) balance "
                        #" ,date_diff('day',date_trunc('day',A.dateTime), date_trunc('day',B.dateTime)) datediff "
                   "From fraxBorrowsDb A "
                            "Join fraxRepaysDb B on A.actions_root_id = B.actions_root_id"
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   " group by 1 "
                   " limit 100 "
                   ).df()


fraxLoanLengthDB.head(10)


BinderException: Binder Error: Table "A" does not have a column named "actions_amountUSD"

In [73]:
vestaDailySnapshot = vesta.Query.marketDailySnapshots(
    orderBy='timestamp',
    orderDirection='desc',
    first=10,
    #skip = skipValue,
    where={              'market':'0xc413dd03555f3eb29d834b482d386b2999dc2eb0'            } #gOHM market
    #where={              'market':'0xfccc27aabd0ab7a0b2ad2b7760037b1eab61616b'            } #DAI for testing
    #0x0ab87046fbb341d058f17cbc4c1133f25a20a52f
)

vestaDailySnapshotslist = sg.query_df([
    vestaDailySnapshot.protocol.name
    ,vestaDailySnapshot.protocol.type
    ,vestaDailySnapshot.protocol.cumulativeUniqueUsers
    ,vestaDailySnapshot.protocol.cumulativeUniqueDepositors
    ,vestaDailySnapshot.protocol.cumulativeUniqueBorrowers
    ,vestaDailySnapshot.protocol.cumulativeUniqueLiquidators
    ,vestaDailySnapshot.protocol.cumulativeUniqueLiquidatees
    ,vestaDailySnapshot.protocol.totalValueLockedUSD
    ,vestaDailySnapshot.protocol.protocolControlledValueUSD
    ,vestaDailySnapshot.protocol.cumulativeSupplySideRevenueUSD
    ,vestaDailySnapshot.protocol.cumulativeProtocolSideRevenueUSD
    ,vestaDailySnapshot.protocol.cumulativeTotalRevenueUSD
    ,vestaDailySnapshot.protocol.totalDepositBalanceUSD
    ,vestaDailySnapshot.protocol.cumulativeDepositUSD
    ,vestaDailySnapshot.protocol.cumulativeWithdrawUSD
    ,vestaDailySnapshot.protocol.totalBorrowBalanceUSD
    ,vestaDailySnapshot.protocol.totalBorrowBalanceUSD
    ,vestaDailySnapshot.protocol.cumulativeLiquidateUSD
    ,vestaDailySnapshot.protocol.creditSupply
    ,vestaDailySnapshot.protocol.creditUtilized
    ,vestaDailySnapshot.market.id
    ,vestaDailySnapshot.market.name
    ,vestaDailySnapshot.market.totalValueLockedUSD
    #,vestaDailySnapshot.market.totalControlledValueUSD
    ,vestaDailySnapshot.market.cumulativeSupplySideRevenueUSD
    ,vestaDailySnapshot.market.cumulativeTotalRevenueUSD
    ,vestaDailySnapshot.market.cumulativeSupplySideRevenueUSD
    ,vestaDailySnapshot.market.cumulativeProtocolSideRevenueUSD
    ,vestaDailySnapshot.market.totalDepositBalanceUSD
    ,vestaDailySnapshot.market.cumulativeDepositUSD
    ,vestaDailySnapshot.market.totalBorrowBalanceUSD
    ,vestaDailySnapshot.market.cumulativeBorrowUSD
    ,vestaDailySnapshot.market.cumulativeLiquidateUSD
    ,vestaDailySnapshot.market.creditSupply
    ,vestaDailySnapshot.market.creditUtilized
    ,vestaDailySnapshot.timestamp
])

vestaDailySnapshotslistTimed= db.query("select "
                   "    * "
                        " ,to_timestamp(marketDailySnapshots_timestamp) dateTime "
                   "From vestaDailySnapshotslist "
                   #"where lower(liquidityPools_name) like lower('%olympus%') "
                    #    "or lower(pairs_token1_symbol) like lower('%OHM%')  "
                   #" limit 100 "
                   ).df()
vestaDailySnapshotslistTimed.drop_duplicates
vestaDailySnapshotslistTimed.head(1000)

KeyError: 'TypeMeta.ObjectMeta.field: no field named cumulativeWithdrawUSD for interface LendingProtocol'

In [97]:
#list of borrowers

borrowerList=db.query(
    "select distinct actions_root_id as wallet_address,'frax' as protocol from fraxBorrowsDb "
    "UNION ALL "
    "select distinct borrows_account_id as wallet_address , 'vesta' as protocol  from vestaBorrowsDbTimed "
    "UNION ALL "
    "select distinct transaction_account_id as wallet_address, 'silo' as protocol from borrowsDbTimed "
).df()

borrowerList.head(1000)


,wallet_address,protocol
0,0xc0b1f200aa9546ce074397bed7438bb2ec033877,frax
1,0xb4fb31e7b1471a8e52dd1e962a281a732ead59c1,frax
2,0xf5b4635f3858a2bc80a17bd4450e05a35ad99a72,frax
3,0x838c80b3fe376b4043781507d76f57c6db0d7d19,frax
4,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,frax
...,...,...
507,0x44504ba8d43b30e882503948654e247b928bcc7f,silo
508,0x1ba67f30e25fc43863b50047d7fd4bd63b9c59ec,silo
509,0x3985caf03eef8f885edb1f8358143bdf08d8d48b,silo
510,0xd3064611b24eba7a6dbcbc26c1b35dfee1170880,silo


In [98]:
borrowerListLength = len(borrowerList)
path =file+'/borrower_list_'+str(dt.datetime.today().strftime("%b-%d-%Y"))+"_"+str(borrowerListLength)+'_records.csv'
borrowerList.to_csv(path, index = False)